# An Implementation of ChatGPT via Gradio Interface

The code in the first block of this notebook is executed when the `app.py` file is run. This notebook is used for the simplicity of interactive development and testing.

In [ ]:
import time
import gradio as gr
from openai import OpenAI


class GPT:
    """Instantiate GPT model for a multi-step conversation."""

    def __init__(self):
        """Setup model parameters and system prompt."""
        self.client = OpenAI()
        self.messages = []

    def chat(self, user_prompt, chat_history, system_prompt, *params):
        """Generate a response with a given parameters."""
        # update GPT parameters -- gradio does not allow dict inputs :(
        params = {
            'model': params[0],
            'temperature': params[1],
            'top_p': params[2],
            'frequency_penalty': params[3],
            'presence_penalty': params[4]
        }

        # update the message buffer
        self.messages = [{'role': 'system', 'content': f'{system_prompt}'}]
        for prompt, response in chat_history:
            self.messages.append({'role': 'user', 'content': f'{prompt}'})
            self.messages.append({'role': 'assistant', 'content': f'{response}'})
        self.messages.append({'role': 'user', 'content': f'{user_prompt}'})

        # generate a response
        completion = self.client.chat.completions.create(
            messages=self.messages,
            **params
        )
        response = completion.choices[0].message.content
        self.messages.append({'role': 'assistant', 'content': response})
        chat_history.append((user_prompt, response))

        # save chat history
        chat_name = save_chat(chat_history)

        return '', chat_history, gr.DownloadButton('Save Chat', value=chat_name)


# TODO: save the model version instead of BOT
def save_chat(chat_history):
    """Write chat history to a file."""
    chat_name = f'/tmp/chat_{int(time.time())}.txt'
    with open(chat_name, 'w', encoding='utf-8') as chat_file:
        for user, bot in chat_history:
            chat_file.write(f'USER: {user}\n BOT: {bot}\n')
    return chat_name


def download_chat():
    """Download chat history in a text file."""
    return gr.DownloadButton(label='Save Chat')


def run_chatbot(gpt):
    """Configure and launch the chatbot interface."""
    with gr.Blocks(title='ChatGPT') as chatbot:

        # chatbot interface
        chat_history = gr.Chatbot(
            height=500,
            layout='bubble',
            label='ChatGPT',
            bubble_full_width=False,
            show_copy_button=True
        )

        with gr.Row():
            user_prompt = gr.Textbox(
                placeholder='Message ChatGPT...',
                container=False,
                min_width=500,
                scale=9
            )
            submit_button = gr.Button(
                value='Submit',
                min_width=300
            )
            save_button = gr.DownloadButton(
                value='Save Chat',
                min_width=300
            )

        # parameters accordion
        with gr.Accordion(label='GPT Parameters', open=False):
            info = gr.Markdown(
                'For parameter documentation see [OpenAI Chat API Reference]' \
                + '(https://platform.openai.com/docs/api-reference/chat)'
            )

            with gr.Row():
                system_prompt = gr.Textbox(
                    value='You are a helpful assistant.',
                    label='system prompt',
                    scale=9
                )
                model_choice = gr.Dropdown(
                    value='gpt-4o',
                    choices=['gpt-4o', 'gpt-4o-mini'],
                    label='model',
                    scale=1
                )

            with gr.Row():
                temperature = gr.Slider(
                    minimum=0.,
                    maximum=2.,
                    value=1.,
                    step=.1,
                    min_width=200,
                    label='temperature'
                )
                top_p = gr.Slider(
                    minimum=0.,
                    maximum=1.,
                    value=1.,
                    step=.01,
                    min_width=200,
                    label='top_p'
                )
                frequency_penalty = gr.Slider(
                    minimum=-2.,
                    maximum=2.,
                    value=0,
                    step=.1,
                    min_width=200,
                    label='frequency_penalty'
                )
                presence_penalty = gr.Slider(
                    minimum=-2.,
                    maximum=2.,
                    value=0,
                    step=.1,
                    min_width=200,
                    label='presence_penalty'
                )

        # submit user prompt and parameters
        params = [model_choice, temperature, top_p, frequency_penalty, presence_penalty]
        inputs = [user_prompt, chat_history, system_prompt, *params]
        outputs = [user_prompt, chat_history, save_button]
        submit_button.click(gpt.chat, inputs=inputs, outputs=outputs)
        user_prompt.submit(gpt.chat, inputs=inputs, outputs=outputs)

        # save chat button
        save_button.click(download_chat, inputs=None, outputs=save_button)

    # instantiate the chatbot
    gr.close_all()
    chatbot.queue(default_concurrency_limit=None)
    chatbot.launch()


if __name__ == '__main__':
    gpt = GPT()
    run_chatbot(gpt)
